Getting Started

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [5]:
import os
import pandas as pd
import json
from tqdm import tqdm
from datasets import Dataset 
from natsort import natsorted

root_dir = "../phogpt/newfile"

mx = 0

cnt_file = 0
texts = []
labels = []
max_len_label = 125
max_len_input = 500

for pth in tqdm(natsorted(os.listdir(root_dir))):
    path_file = os.path.join(root_dir, pth)
    with open(path_file, "r", encoding="utf-8") as f:
        data = f.read()
    f.close()

    text, label = data.split("\n*****\n")
    if len(text) > 600 or len(text) < 50:
        continue

    # preprocessing the data
    text = text.replace("\n","")
    text = text.split(" ")
    if len(text) > max_len_input:
        text = text[:max_len_input]
        for subtext in reversed(range(len(text))):
            if "." in text[subtext]:
                text = text[:subtext + 1]
                break
        text = " ".join(text)

        if len(text) < 100:
            text = " ".join(text)
    else:
        text = " ".join(text)

    label = label.split(" ")

    if len(label) > max_len_label:
        label = " ".join(label[:max_len_label])
    else:
        label = " ".join(label)
    
    text = "Hãy tóm tắt câu sau: "+text 

    file_save = "short_texts/texts/"+str(cnt_file)+".txt"
    with open(file_save, "w", encoding = "utf-8") as fs:
        fs.write(text)
    cnt_file+=1
    fs.close()

    texts.append(text)
    labels.append(label)

print(len(texts))
print(len(labels))
df = pd.DataFrame({"prompt": texts, "output": labels}) 
dataset = Dataset.from_dict(df)

100%|██████████| 7228/7228 [00:00<00:00, 25074.40it/s]


3986
3986


In [ ]:
from datasets import Dataset

# Assuming `dataset` is your Dataset object
dataset = dataset.map(lambda example: {'text':  example['prompt']  + "###Tóm tắt: "+ example['output'] +"***END"})

Loading the model

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

# model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
# model_name = "vilm/vinallama-2.7b-chat"
# model_name = "vinai/PhoGPT-7B5-Instruct"
model_name = "bigscience/bloom-1b1"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Setting up training arguments

In [6]:
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

output_dir = "./bloom1b1"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 5000
num_train_epochs = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 30000
warmup_ratio = 0.03
lr_scheduler_type = "linear"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Setting up the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 2096

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

Start the finetuning

In [8]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ndtoan-10a10kx (inresai). Use `wandb login --relogin` to force relogin


  0%|          | 0/30000 [00:00<?, ?it/s]

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.1822, 'learning_rate': 1.777777777777778e-06, 'epoch': 0.01}
{'loss': 2.077, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}
{'loss': 2.1086, 'learning_rate': 6e-06, 'epoch': 0.02}
{'loss': 1.9998, 'learning_rate': 8.222222222222223e-06, 'epoch': 0.03}
{'loss': 1.8452, 'learning_rate': 1.0444444444444445e-05, 'epoch': 0.03}
{'loss': 2.0735, 'learning_rate': 1.2666666666666668e-05, 'epoch': 0.04}
{'loss': 2.0943, 'learning_rate': 1.4888888888888888e-05, 'epoch': 0.05}
{'loss': 2.0095, 'learning_rate': 1.7111111111111112e-05, 'epoch': 0.05}
{'loss': 2.1143, 'learning_rate': 1.9333333333333333e-05, 'epoch': 0.06}
{'loss': 1.8284, 'learning_rate': 2.1555555555555555e-05, 'epoch': 0.07}
{'loss': 1.9527, 'learning_rate': 2.377777777777778e-05, 'epoch': 0.07}
{'loss': 2.1182, 'learning_rate': 2.6000000000000002e-05, 'epoch': 0.08}
{'loss': 2.0324, 'learning_rate': 2.8000000000000003e-05, 'epoch': 0.09}
{'loss': 2.1708, 'learning_rate': 3.0222222222222225e-05, 'epoch': 0.09}


TrainOutput(global_step=30000, training_loss=0.610537412100037, metrics={'train_runtime': 61559.6235, 'train_samples_per_second': 0.975, 'train_steps_per_second': 0.487, 'train_loss': 0.610537412100037, 'epoch': 20.15})

Saving the model

In [9]:
# model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
# model_to_save = trainer.model
model.save_pretrained("outputs")

Quickly testing inference

In [11]:
text = dataset[3]["prompt"]
text

'Hãy tóm tắt câu sau: Điều tra vụ cháy tàu trên sông biên giới Ka Long.\nTối 4.1, thông tin từ UBND TP.Móng Cái cho biết, các cơ quan chức năng của địa phương này đang vào cuộc điều tra vụ cháy tàu trên sông biên giới Ka Long. 3 tàu du lịch bốc cháy ngùn ngụt L.V Trước đó, vào khoảng 18 giờ 30 phút ngày 4.1, trên sông biên giới Ka Long, đoạn qua P.Ninh Dương (TP.Móng Cái) 3 tàu gỗ đang neo đậu bất ngờ xảy ra cháy. Theo một số người dân cho biết, vào thời điểm trên, 3 tàu gỗ du lịch của công ty du lịch Long Dương, hiện đã ngừng hoạt động neo đậu tại cảng Phương Oanh (khu Hạ Long, P.Ninh Dương, TP.Móng Cái) bốc khói đen nghi ngút, ít phút sau lửa bốc cháy ngùn ngụt lan rộng. Hàng chục cán bộ chiến sĩ công an, quân đội tới hiện trường dập lửa L.V Nhận được thông tin vụ việc, chính quyền TP.Móng Cái đã huy động hàng chục cán bộ chiến sĩ, người dân có mặt tại hiện trường dập lửa. Đến khoảng 21 giờ cùng ngày đám cháy đã được dập tắt và không gây thiệt hại nào về người. Rất may vụ cháy tàu kh

In [12]:
text = """
TP.HCM cần 43 giáo viên từ sinh viên xuất sắc, bao nhiêu ứng viên trúng tuyển?.
Đợt tuyển dụng giáo viên cho năm học 2023-2024 của Sở GD-ĐT TP.HCM diễn ra vào tháng 6.2023. Theo đó, căn cứ vào quyết định phê duyệt của Chủ tịch UBND TP.HCM về kết quả tuyển dụng viên chức thành phố từ nguồn sinh viên xuất sắc, cán bộ khoa học trẻ năm 2023, Sở GD-ĐT TP.HCM đã có thông báo kết quả tuyển giáo viên năm 2023. Cụ thể, có 2 sinh viên xuất sắc chuyên ngành sư phạm toán và sư phạm tiếng Anh đã trúng tuyển theo hình thức tuyển thẳng vị trí giáo viên THPT. Với kết quả trên, 2 ứng viên trúng tuyển sẽ hoàn thiện hồ sơ tuyển dụng trong vòng 30 ngày sau đó Sở GD-ĐT sẽ phân công nhiệm sở.
Được biết, đây là lần đầu tiên Sở GD-ĐT TP.HCM xét tuyển thẳng giáo viên từ nguồn sinh viên tốt nghiệp xuất sắc, cán bộ khoa học trẻ. Được biết tổng nhu cầu tuyển dụng viên chức trong các đơn vị giáo dục công lập từ nguồn sinh viên tốt nghiệp xuất sắc là 43 chỉ tiêu các chuyên ngành sư phạm: tin học, tiếng Anh, âm nhạc, mỹ thuật, toán học, vật lý, hóa học, sinh học, ngữ văn, lịch sử, địa lý, giáo dục thể chất, giáo dục quốc phòng-an ninh.
Theo đánh giá của một cán bộ phụ trách công tác tổ chức cán bộ của Sở GD-ĐT, số lượng ứng viên đủ điều kiện xét tuyển theo chính sách này còn hạn chế do ràng buộc nhiều quy định. Chẳng hạn như ứng viên tốt nghiệp ĐH loại xuất sắc nhưng chỉ cần một học kỳ không đạt cũng trượt vòng sơ tuyển. Ngoài ra, trong thời gian học cấp THPT, ứng viên phải có học lực giỏi, đạt giải ba trở lên tại các kỳ thi học sinh giỏi cấp tỉnh, quốc gia…
"""

In [14]:
device = "cuda:0"

prompt = text
inputs = tokenizer(prompt, return_token_type_ids = False, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=300, top_k=50,  top_p=1, temperature = 0.8)
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = output.split("###Tóm tắt:")[1]
print(response)

/home/inresai/anaconda3/envs/llminresai/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 Sở GD-ĐT TP.HCM đang đối mặt với tình trạng tuyển dụng viên chức và sinh viên không đạt yêu cầu. Đã có 382 thí sinh tham gia so với năm ngoái, trong đó có 382 thí sinh dự thi học sinh giỏi cấp quốc gia. Sở GD-ĐT đang đối mặt với tình trạng tuyển dụng viên chức và sinh viên không đạt yêu cầu. Cơ quan GD-ĐT TP.HCM cũng đang xem xét việc tuyển dụng viên chức của các thí sinh và quy định rằng một số giải pháp có thể xuất hiện trong tương lai.***END
